<a href="https://colab.research.google.com/github/yuuuwoo/Vehicle-Detection/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.chdir('/content/drive/MyDrive/Project')

In [ ]:
!git clone https://github.com/zzh142857/SiameseFC-tf.git

fatal: destination path 'SiameseFC-tf' already exists and is not an empty directory.


In [ ]:
os.chdir('/content/drive/MyDrive/Project/SiameseFC-tf')

In [ ]:
import numpy as np
import cv2
import os
import os.path
from src.region_to_bbox import region_to_bbox_normalized
from random import shuffle


def prepare_shuffled_list(data, output, directory, num):
    """
        Input:
            data_folder: relative path of folder who contains all the vedio folders for training.
            tfrecord_name: nameof the tfrecord file
            output_directory: relative dir which will contain the tfrecord file
    """
    if not os.path.exists(directory) or os.path.isfile(directory):
        os.makedirs(directory)
        
    
        
    Dir = os.getcwd()
    data = os.path.join(Dir, data)
    #get a list of dirs in data_folder, in each of which contains a training vedio
    vfolderlist = sorted([dir for dir in os.listdir(data) if not os.path.isfile(os.path.join(data, dir))])[:num]
    vindex = 0
    output_list = []

    #loop through all the vedio folders
    for vedio_folder in vfolderlist:
        print("Reading images from " + vedio_folder);
        vindex += 1
        
        vedio_folder = os.path.join(data, vedio_folder)
        #get a list of dirs in data_folder, in each of which contains a training vedio
        file_list = [dir for dir in os.listdir(os.path.join(vedio_folder,'img')) if os.path.isfile(os.path.join(os.path.join(vedio_folder,'img'), dir))]
        img_list = sorted([file for file in file_list if file.endswith(".jpg")])
        print(len(img_list))
        
        gtfilename = 'groundtruth_rect.txt'        
        print(vedio_folder,gtfilename)
        if vedio_folder=='/content/drive/MyDrive/Project/SiameseFC-tf/data/Human4':
          gt_file_name='groundtruth_rect.2.txt'
        assert os.path.exists(os.path.join(vedio_folder, gtfilename))
        
        gt_file = open(os.path.join(vedio_folder, gtfilename), 'r')


        gts = gt_file.readlines()
        for i in range(len(gts)):
          line=gts[i].replace('\t',',')
          gts[i]=line
        print(len(gts))
        #check if num of ground truth equals the num of img files
        if len(gts)>len(img_list):
          gts=gts[:len(img_list)]
        elif len(img_list)>len(gts):
          img_list=img_list[:len(gts)]
        assert len(gts) == len(img_list)
        _examples = list(zip(img_list, gts))
        
        #prepare examplar z
        z = os.path.join(os.path.join(vedio_folder, 'img'), img_list[0])
        z_img = cv2.imread(z)
        z_gt = gts[0].strip("\n").split(",") 
        print(z_gt)  
        assert len(z_gt) == 4
        #x, y in ground truth is the coordinate of the topleft corner, we need to convert to the center
        z_pos_x, z_pos_y, z_target_w, z_target_h = region_to_bbox_normalized(z_gt, z_img.shape[1], z_img.shape[0])
               
        for _example in _examples[1: ]:
            assert len(_example) == 2
            gt = _example[1].strip("\n").split(",") 
            assert len(gt) == 4
            
            x = os.path.join(os.path.join(vedio_folder, 'img'), _example[0])
            x_img = cv2.imread(x)                                    
            x_pos_x, x_pos_y, x_target_w, x_target_h = region_to_bbox_normalized(gt, x_img.shape[1], x_img.shape[0]) 
            
            
            output_list.append(z + " " + x + " " + str(z_pos_x)+ " "+ str(z_pos_y)+ " "+  str(z_target_w)+ " "+ str(z_target_h)+ " "+  str(x_pos_x)+ " "+ str(x_pos_y)+ " "+  str(x_target_w)+ " "+  str(x_target_h))
            z = x
            z_pos_x, z_pos_y, z_target_w, z_target_h = x_pos_x, x_pos_y, x_target_w, x_target_h
    print("Finished loading all the training data. Start shuffling...")
    shuffle(output_list);
    with open(os.path.join(directory,output) + ".txt","w+") as f:
        for output_file in output_list:
            f.write(output_file + "\n") 
    print("Done. tfrecords file saved at " + directory)
        

    
if __name__ == "__main__":
    prepare_shuffled_list("data", "shuffled_data_list", "tfrecords", num = 1)

Reading images from Basketball
725
/content/drive/MyDrive/Project/SiameseFC-tf/data/Basketball groundtruth_rect.txt
725
['198', '214', '34', '81']
Finished loading all the training data. Start shuffling...
Done. tfrecords file saved at tfrecords


In [ ]:
import tensorflow as tf
import cv2
import os
import os.path
from src.parse_arguments import parse_arguments


def transform2tfrecord(data_file, tfrecord_name, output_directory, resize_width, resize_height):
    """
        Input:
            data_folder: relative path of folder who contains all the vedio folders for training.
            tfrecord_name: nameof the tfrecord file
            output_directory: relative dir which will contain the tfrecord file

    """
    if not os.path.exists(output_directory) or os.path.isfile(output_directory):
        os.makedirs(output_directory)
        
    #init tfrecord file
    file_name = os.path.join(output_directory, tfrecord_name + '.tfrecords')
    writer = tf.io.TFRecordWriter(file_name)
        
    cur_dir = os.getcwd()
    data_folder = os.path.join("tfrecords", data_file)
    with open(data_folder, "r") as f:
        print("Generating tfrecord from " + data_folder)
        data_list = f.readlines()
        for data in data_list:
            z, x, z_pos_x, z_pos_y, z_target_w, z_target_h, x_pos_x, x_pos_y, x_target_w, x_target_h = data.strip("\n ").split(" ")
            z_pos_x = float(z_pos_x)
            z_pos_y = float(z_pos_y)
            z_target_w = float(z_target_w)
            z_target_h = float(z_target_h)
            x_pos_x = float(x_pos_x)
            x_pos_y = float(x_pos_y)
            x_target_w = float(x_target_w)
            x_target_h = float(x_target_h)
            z_img = cv2.imread(z)
            z_img = cv2.resize(z_img, (resize_width,resize_height))
            x_img = cv2.imread(x)
            x_img = cv2.resize(x_img, (resize_width,resize_height))
            z_raw = z_img.tostring()
            x_raw = x_img.tostring()
            

            example = tf.train.Example(features=tf.train.Features(feature={
                'z_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[z_raw])),
                'x_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[x_raw])),
                'z_pos_x': tf.train.Feature(float_list=tf.train.FloatList(value=[z_pos_x])),
                'z_pos_y': tf.train.Feature(float_list=tf.train.FloatList(value=[z_pos_y])),
                'z_target_w': tf.train.Feature(float_list=tf.train.FloatList(value=[z_target_w])),
                'z_target_h': tf.train.Feature(float_list=tf.train.FloatList(value=[z_target_h])),
                'x_pos_x': tf.train.Feature(float_list=tf.train.FloatList(value=[x_pos_x])),
                'x_pos_y': tf.train.Feature(float_list=tf.train.FloatList(value=[x_pos_y])),
                'x_target_w': tf.train.Feature(float_list=tf.train.FloatList(value=[x_target_w])),
                'x_target_h': tf.train.Feature(float_list=tf.train.FloatList(value=[x_target_h]))
                
            }))
            writer.write(example.SerializeToString())
        
        
 
        
    writer.close()
    print("Writer closed.")
    print(tfrecord_name + '.tfrecords'+" is written to "+output_directory)



    
if __name__ == "__main__":
    hp, evaluation, run, env, design = parse_arguments()
    transform2tfrecord("shuffled_data_list.txt", "training_dataset", "tfrecords", resize_width = design.resize_width, resize_height = design.resize_height)

('/usr/local/lib/python3.7/dist-packages/keras', 'tensorflow/python')
Generating tfrecord from tfrecords/shuffled_data_list.txt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


Writer closed.
training_dataset.tfrecords is written to tfrecords


In [ ]:
from __future__ import division
import sys
import os
os.chdir('/content/drive/MyDrive/Project/SiameseFC-tf')
import src.siamese as siam
from src.trainer import trainer
from src.parse_arguments import parse_arguments
import src.read_training_dataset


"""
    training procedure:
    1,input z, x, pos_x, pos_y, w, d and gt of x
    2,pad and crop z,x
    3,calculate score map
    4,calculate loss
    5,bp, update variable
"""


def train():
    # avoid printing TF debugging information
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    # TODO: allow parameters from command line or leave everything in json files?
    hp, evaluation, run, env, design = parse_arguments()
    # Set size for use with tf.image.resize_images with align_corners=True.
    # For example,
    #   [1 4 7] =>   [1 2 3 4 5 6 7]    (length 3*(3-1)+1)
    # instead of
    # [1 4 7] => [1 1 2 3 4 5 6 7 7]  (length 3*3)
    final_score_sz = hp.response_up * (design.score_sz - 1) + 1
    print(design.batch_size)
    
    # build the computational graph of Siamese fully-convolutional network
    siamNet = siam.Siamese(design.batch_size)
    # get tensors that will be used during training
    image, z_crops, x_crops, templates_z, scores, loss, train_step, distance_to_gt, summary= siamNet.build_tracking_graph_train(final_score_sz, design, env, hp)
 
    # read tfrecodfile holding all the training data
    datareader = src.read_training_dataset.myReader(design.resize_width, design.resize_height, design.channel)
    batcheddata = datareader.read_tfrecord(os.path.join(env.tfrecord_path, env.tfrecord_filename), num_epochs = design.num_epochs, batch_size = design.batch_size)
    
    # run trainer
    trainer(hp, run, design, final_score_sz, batcheddata, image, templates_z, scores, loss, train_step, distance_to_gt,  z_crops, x_crops, siamNet, summary)




if __name__ == '__main__':
    sys.exit(train())

Instructions for updating:
non-resource variables are not supported in the long term
Using Tensorflow 2.8.0
8
shape of single_crops_x:  (3, 255, 255, 3) shape of x_crops:  (24, 255, 255, 3)
shape of single_crops_z:  (127, 127, 3) shape of z_crops:  (8, 127, 127, 3)
> Layer 1
		MAX-POOL: size 3 and stride 2
> Layer 2
		MAX-POOL: size 3 and stride 1
> Layer 3
> Layer 4
> Layer 5


/content/drive/MyDrive/Project/SiameseFC-tf/src/convolutional.py:54: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  h = tf.compat.v1.layers.batch_normalization(h)
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/normalization.py:463: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs, training=training)


shape of template_z: (8, 17, 17, 32)
shape of templates_z: [24, 17, 17, 32]
shape of templates_x: [24, 49, 49, 32]
shape_net_z: (24, 17, 17, 32)
shape of net: [1, 33, 33, 768]
shape of net_cat: [24, 33, 33, 32]
shape of net_final: [24, 33, 33, 1]
shape of big score map: [24, 257, 257, 1]


/content/drive/MyDrive/Project/SiameseFC-tf/src/siamese.py:204: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net_final = tf.layers.batch_normalization(net_final)


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
Prefer Dataset.range instead.
Instructions for updating:
Prefer Dataset.range instead.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based inp

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


##After training the final loss is 0.602622